In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
outputevents_df = pd.read_csv(dataDirStr + "OUTPUTEVENTS.csv")
outputevents_df.columns = outputevents_df.columns.str.lower()
outputevents_df

,row_id,subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom,storetime,cgid,stopped,newbottle,iserror
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,4340476,68375,123645.0,276251.0,2175-09-05 23:00:00,226588,30.0,mL,2175-09-05 22:48:00,16915,NaN,NaN,NaN
4349214,4340477,68375,123645.0,276251.0,2175-09-06 00:00:00,226588,0.0,mL,2175-09-06 00:03:00,20971,NaN,NaN,NaN
4349215,4340478,68375,123645.0,276251.0,2175-09-06 01:00:00,226588,40.0,mL,2175-09-06 01:13:00,20971,NaN,NaN,NaN
4349216,4340479,68375,123645.0,276251.0,2175-09-06 02:00:00,226588,20.0,mL,2175-09-06 02:14:00,20971,NaN,NaN,NaN


`OUTPUTEVENTS` table is the output data from the CareVue and Metavision ICU database. Outputs are fluids which have either been excreted by the patient, such as urine output, or extracted from the patient, for example through a drain.

`itemid` is the identifier for a single measurement type in the database. Multiple rows can have the same `itemid`.

Refer to `D_ITEMS` table for exact definitions of each `itemid`.

In [4]:
c = outputevents_df.itemid.value_counts()[:5]
p = outputevents_df.itemid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
40055,1917421,44.09
226559,1186717,27.29
40076,152716,3.51
43175,108982,2.51
40054,81828,1.88


`value` and `valueuom` list the amount of a substance `itemid` given at the `charttime`.

`storetime` records the time at which an observation was manually input or manually validated by a member of clinical staff.

`cgid` is the caregiver that recorded the data

In [5]:
c = outputevents_df.cgid.value_counts()[:5]
p = outputevents_df.cgid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
21570,248290,5.71
14988,31902,0.73
19611,30454,0.70
15803,27243,0.63
19963,27046,0.62


`stopped` indicates if the order was disconnected at the given `charttime`

`newbottle` indicates that a new bag of solution was given at `charttime`

`iserror` happens when a caregiver specifies that the observation is an error with no other details provided.

However, these last three are not used.